### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
print(today)

2025-02-11


In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'Today: {today}')
print(f'Yesterday: {yesterday}')

Today: 2025-02-11
Yesterday: 2025-02-10


In [7]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [9]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [11]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"OSD path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
OSD path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [13]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-02-10'
ORDER BY a.name



(20, 1)

In [15]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-02-10' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [18]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-02-06,2025-02-10,5.40,5.40,46902358,5.45,5.35,B,SET,0.00,0.00
1,AIMIRT,2025-02-07,2025-02-10,10.50,10.50,294770,10.50,10.40,B,SET,0.00,0.00
2,CPNREIT,2025-02-10,2025-02-10,11.90,12.00,6433323,12.30,11.90,B,SET,0.10,0.84
3,DIF,2025-02-05,2025-02-10,8.45,8.40,43070000,8.50,8.35,B,SET,-0.05,-0.59
4,GVREIT,2025-02-03,2025-02-10,6.60,6.60,3147510,6.75,6.55,B,SET,0.00,0.00
5,JMART,2025-02-07,2025-02-10,10.20,9.80,14575828,10.20,9.60,B,SET50,-0.40,-3.92
6,KCE,2025-02-07,2025-02-10,20.30,19.80,12807640,20.30,19.60,B,SET100,-0.50,-2.46
7,MCS,2025-01-29,2025-02-10,6.25,5.85,2944458,6.30,5.80,B,SET,-0.40,-6.40
8,NER,2025-02-07,2025-02-10,4.74,4.62,15111252,4.74,4.58,B,SET,-0.12,-2.53
9,SENA,2025-01-29,2025-02-10,2.18,2.14,8073396,2.20,2.12,B,SET,-0.04,-1.83


In [20]:
df.shape

(20, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [24]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,3BBIF,2025-02-06,2025-02-10,5.40,5.40,"46,902,358",5.45,5.35,0.00%,B
1,AIMIRT,2025-02-07,2025-02-10,10.50,10.50,"294,770",10.50,10.40,0.00%,B
2,CPNREIT,2025-02-10,2025-02-10,11.90,12.00,"6,433,323",12.30,11.90,0.84%,B
3,DIF,2025-02-05,2025-02-10,8.45,8.40,"43,070,000",8.50,8.35,-0.59%,B
4,GVREIT,2025-02-03,2025-02-10,6.60,6.60,"3,147,510",6.75,6.55,0.00%,B
5,JMART,2025-02-07,2025-02-10,10.20,9.80,"14,575,828",10.20,9.60,-3.92%,B
6,KCE,2025-02-07,2025-02-10,20.30,19.80,"12,807,640",20.30,19.60,-2.46%,B
7,MCS,2025-01-29,2025-02-10,6.25,5.85,"2,944,458",6.30,5.80,-6.40%,B
8,NER,2025-02-07,2025-02-10,4.74,4.62,"15,111,252",4.74,4.58,-2.53%,B
9,SENA,2025-01-29,2025-02-10,2.18,2.14,"8,073,396",2.20,2.12,-1.83%,B


In [26]:
file_name = "daily-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [28]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\daily-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales.csv


In [30]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [32]:
sales = df[cols]
sales.shape

(20, 10)

In [34]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [36]:
prices = pd.read_csv(input_file)
prices.shape

(182, 10)

In [38]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(20, 19)

In [40]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,3BBIF,2025-02-06,2025-02-10,5.40,5.40,46902358,5.45,5.35,0.00,B,5.45,0.05,8414482,2025-02-11
1,AIMIRT,2025-02-07,2025-02-10,10.50,10.50,294770,10.50,10.40,0.00,B,10.50,0.00,64123,2025-02-11
2,CPNREIT,2025-02-10,2025-02-10,11.90,12.00,6433323,12.30,11.90,0.84,B,12.00,0.00,966300,2025-02-11
3,DIF,2025-02-05,2025-02-10,8.45,8.40,43070000,8.50,8.35,-0.59,B,8.45,0.05,12999223,2025-02-11
4,GVREIT,2025-02-03,2025-02-10,6.60,6.60,3147510,6.75,6.55,0.00,B,6.65,0.05,197110,2025-02-11
5,JMART,2025-02-07,2025-02-10,10.20,9.80,14575828,10.20,9.60,-3.92,B,10.00,0.20,10660930,2025-02-11
6,KCE,2025-02-07,2025-02-10,20.30,19.80,12807640,20.30,19.60,-2.46,B,21.20,1.40,13788336,2025-02-11
7,MCS,2025-01-29,2025-02-10,6.25,5.85,2944458,6.30,5.80,-6.40,B,6.25,0.40,198223,2025-02-11
8,NER,2025-02-07,2025-02-10,4.74,4.62,15111252,4.74,4.58,-2.53,B,4.70,0.08,4662442,2025-02-11
9,SENA,2025-01-29,2025-02-10,2.18,2.14,8073396,2.20,2.12,-1.83,B,2.14,0.00,276101,2025-02-11


In [42]:
file_name = "daily-sales-prices.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [44]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\daily-sales-prices.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales-prices.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales-prices.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales-prices.csv


In [46]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(god_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(icd_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [49]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [51]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
3BBIF    2025-02-06 2025-02-11   5.40   5.45   0.93%  46,902,358   5.45   5.35 B Update
AIMIRT   2025-02-07 2025-02-11  10.50  10.50    0.0%     294,770  10.50  10.40 B Update
AWC      2025-02-11 2025-02-11   2.94   3.00   2.04% 133,704,776   3.00   2.88 S Insert
BCH      2025-02-11 2025-02-11  13.40  14.20   5.97%   6,825,676  14.20  13.40 S Insert
CPNREIT  2025-02-10 2025-02-11  11.90  12.00   0.84%   6,433,323  12.30  11.90 B Update
DIF      2025-02-11 2025-02-11   8.40   8.45    0.6%  43,070,000   8.50   8.35 B Insert
GVREIT   2025-02-03 2025-02-11   6.60   6.65   0.76%   3,147,510   6.75   6.55 B Update
JMART    2025-02-11 2025-02-11   9.80  10.00   2.04%  14,575,828  10.20   9.60 B Insert
JMT      2025-02-11 2025-02-11  13.80  14.10   2.17%  27,156,122  14.30  13.40 S Insert
KCE      2025-02-11 2025-02-11  

In [53]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [56]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % today
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-02-11' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [58]:
df = pd.read_sql(sql, conlite)
df.shape

(20, 10)

In [60]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-02-06,2025-02-11,5.40,5.45,"93,804,716",5.45,5.35,B,SET,0.05,0.93%
1,AIMIRT,2025-02-07,2025-02-11,10.50,10.50,"589,540",10.50,10.40,B,SET,0.00,0.00%
2,CPNREIT,2025-02-10,2025-02-11,11.90,12.00,"12,866,646",12.30,11.90,B,SET,0.10,0.84%
3,DIF,2025-02-11,2025-02-11,8.40,8.45,"43,070,000",8.50,8.35,B,SET,0.05,0.60%
4,GVREIT,2025-02-03,2025-02-11,6.60,6.65,"6,295,020",6.75,6.55,B,SET,0.05,0.76%
5,JMART,2025-02-11,2025-02-11,9.80,10.00,"14,575,828",10.20,9.60,B,SET50,0.20,2.04%
6,KCE,2025-02-11,2025-02-11,19.80,21.20,"12,807,640",21.20,19.60,B,SET100,1.40,7.07%
7,MCS,2025-02-11,2025-02-11,5.85,6.25,"2,944,458",6.30,5.80,B,SET,0.40,6.84%
8,NER,2025-02-11,2025-02-11,4.62,4.70,"15,111,252",4.74,4.58,B,SET,0.08,1.73%
9,SENA,2025-01-29,2025-02-11,2.18,2.14,"16,146,792",2.20,2.12,B,SET,-0.04,-1.83%


In [62]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [64]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
17,SYNEX,2025-02-07,2025-02-11,11.30,10.50,"5,769,312",11.40,10.50,S,SET,-0.80,-7.08%


In [66]:
df_sort.query('percent >= 5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
13,BCH,2025-02-11,2025-02-11,13.40,14.20,"6,825,676",14.20,13.40,S,SET100,0.80,5.97%
7,MCS,2025-02-11,2025-02-11,5.85,6.25,"2,944,458",6.30,5.80,B,SET,0.40,6.84%
6,KCE,2025-02-11,2025-02-11,19.80,21.20,"12,807,640",21.20,19.60,B,SET100,1.40,7.07%


In [68]:
conlite.close()

In [70]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-02-11 20:38:25
